In [ ]:
import import_ipynb
from Utils import integracion_obs_infolab, integracion_obs_infolab_difuntos,obtener_columnas_destino, integracionpaciente, crear_df_ficharecolector,conectar_minio,guardar_df_en_minio,generar_ruta_fecha
import psycopg2
import os
from dotenv import load_dotenv

In [ ]:
# Cargar variables del archivo .env
load_dotenv()

In [ ]:
connection_params = {
    'db_name': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port':os.getenv("DB_PORT")
}

In [ ]:
# 2. Obtener columnas destino
columnas_bd = obtener_columnas_destino(connection_params)

INFOLAB

In [ ]:
# 3. Ejecutar integración
if columnas_bd:
    resultadoPaciente = integracionpaciente(columnas_bd, connection_params)
    if resultadoPaciente:
        print("Integración de paciente completada exitosamente.")
        # Abrir conexión y pasarla a la función       
        resutadoInfoLab=integracion_obs_infolab(connection_params)
        if resutadoInfoLab:
            print("Integración de observaciones InfoLab completada exitosamente.")  
        else:
            print("Error al ejecutar la integración de observaciones InfoLab.")
    else:
        print("Error al ejecutar la integración de paciente.")
else:
    print("No se pudieron obtener las columnas destino")

DIFUNTOS

In [ ]:
# 3. Ejecutar integración con infolab difuntos
if columnas_bd:
    resultadoPaciente = integracionpaciente(columnas_bd, connection_params)
    if resultadoPaciente:
        print("Integración de paciente completada exitosamente.")
        # Abrir conexión y pasarla a la función       
        resutadoInfoLab=integracion_obs_infolab_difuntos(connection_params)
        if resutadoInfoLab:
            print("Integración de observaciones InfoLab-difuntos completada exitosamente.")  
        else:
            print("Error al ejecutar la integración de observaciones InfoLab-difuntos.")
    else:
        print("Error al ejecutar la integración de paciente.")
else:
    print("No se pudieron obtener las columnas destino")

MINIO

In [ ]:
df = crear_df_ficharecolector(connection_params)
if df is not None:
    print("DataFrame creado exitosamente.")
else:
    print("Error al crear el DataFrame del fichero recolector.")

In [ ]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

In [ ]:
ruta_fecha = generar_ruta_fecha() 

In [ ]:
# Guardar el DataFrame en MinIO
ruta_minio = guardar_df_en_minio(
    minio_client=minio_client,  # Cliente de la conexión anterior
    df=df,
    bucket_name="yachay-golden",
    ruta_destino=f"Sabana_inicial/{ruta_fecha}/sabana_inicial",  # La extensión se agregará automáticamente
    formato='parquet',  # También puede ser 'csv', 'json' o 'excel'
    crear_bucket=True
)